In [1]:
import pandas as pd
import os
import requests
from PIL import Image
import warnings
import shutil


warnings.filterwarnings("ignore")

In [7]:
#read full training set csv, assign column names and index, save for future. Do not run this cell again
#df3 = pd.read_csv(r'large_meta_data/Image_Labels_Subset_Train_GCC-Labels-training.tsv', sep='\t', header=None, names=['caption', 'link', 'objects','mid', 'object_confidences'])
#print(len(df3))
# #df3.to_csv(r'large_meta_data/Image_Labels_Object_Subset.tsv',index_label='index')

In [6]:
train_gcc_meta = pd.read_csv(r'large_meta_data/Image_Labels_Object_Subset.tsv')
print(len(train_gcc_meta))
train_gcc_meta.head(2)

2007528


,index,caption,link,objects,mid,object_confidences
0,0,christmas tree on a black background .,https://thumb1.shutterstock.com/display_pic_wi...,"christmas tree,christmas decoration,font,text,...","/m/025nd,/m/05fc9mj,/m/03gq5hm,/m/07s6nbt,/m/0...","0.9818305373191833,0.952756941318512,0.9227379..."
1,1,item : drawing of a figure surrounded by person,https://i.pinimg.com/736x/f9/fd/48/f9fd4878090...,"drawing,modern art,line,visual arts,art,sketch...","/m/02csf,/m/015r61,/m/03scnj,/m/0p9xx,/m/0jjw,...","0.8945257067680359,0.8489813804626465,0.828646..."


In [52]:
#save the current working directory to use later
cwd = os.getcwd()
cwd

'C:\\Users\\apra\\Desktop\\SPRING 2020\\DS 4400\\ML1_Final_Proj'

In [26]:
#os.chdir('..')
#os.chdir('ML1_Final_Proj/')

In [28]:
train_gcc_meta.head(2)

,index,caption,link,objects,mid,object_confidences
0,0,christmas tree on a black background .,https://thumb1.shutterstock.com/display_pic_wi...,"christmas tree,christmas decoration,font,text,...","/m/025nd,/m/05fc9mj,/m/03gq5hm,/m/07s6nbt,/m/0...","0.9818305373191833,0.952756941318512,0.9227379..."
1,1,item : drawing of a figure surrounded by person,https://i.pinimg.com/736x/f9/fd/48/f9fd4878090...,"drawing,modern art,line,visual arts,art,sketch...","/m/02csf,/m/015r61,/m/03scnj,/m/0p9xx,/m/0jjw,...","0.8945257067680359,0.8489813804626465,0.828646..."


In [53]:
#we start a dataframe with indices of already sampled data to keep track of the data we already have. This does not include includeabs
#initial 10k samples since the indices were lost.Do not run this cell again

initial_sample = train_gcc_meta.sample(1)
if(not os.path.exists(r'data/small_sample')):
    os.mkdir(r'data/small_sample')
    os.mkdir(r'data/small_sample/images')
os.chdir(r'data/small_sample/images')

#keep track of number of samples sucessfully taken
num_sampled = 0

successfully_sampled_indices = []
dead_link_indices = []

#faster to append to directory list than directly dataframe
cleaned_sample = []

for idx, row in initial_sample.iterrows():
    url = row['link']
    try:
        r = requests.get(url, verify=False, timeout=2, stream=True)
        if r.status_code == 200:
            #append the row to the meta_dataframe for this sample
            print(row)
            cleaned_sample.append(row.to_dict())
            with open(str(idx)+'.png', 'wb') as out_file:
                #download the image and name it with its index in train.gcc
                shutil.copyfileobj(r.raw, out_file)
                #store the successfully sampled image's index
                successfully_sampled_indices.append(idx)
                print(idx)
                num_sampled +=1
        else:
            print(f"failed to download image: {idx}")
            dead_link_indices.append(idx)
            
    except requests.exceptions.ReadTimeout:
        print (f"Read timed out for index: {idx}")
    except Exception as e:
        print(e)
        print (f"Other requests excpetion for index: {idx}")

#save the dataframes

#samples metadata in parent
os.chdir('..')
sample_meta_df = pd.DataFrame.from_dict(cleaned_sample)
sample_meta_df = sample_meta_df.set_index('index')
sample_meta_df.to_csv('meta_data')



#global sampled indices and dead links dataframe in project meta_data folder
os.chdir(cwd)
os.chdir(r'meta_data')
successfully_sampled_indices_df = pd.Series(successfully_sampled_indices)
successfully_sampled_indices_df.to_csv('sampled_idx_list',index = False, header = ['index'])

dead_link_indice_df = pd.Series(dead_link_indices)
dead_link_indice_df.to_csv('dead_link_idx_list', index=False, header = ['index'])


#go back to cwd
os.chdir(cwd)



        





index                                                            121763
caption                                   actor arrives at the premiere
link                  http://media.gettyimages.com/photos/actor-dams...
objects               forehead,premiere,smile,fashion design,caesar cut
mid                   /m/025kyy,/m/03n3f3,/m/019nj4,/m/02qqd4n,/m/06...
object_confidences    0.8217551112174988,0.6771990656852722,0.641064...
Name: 121763, dtype: object
121763


In [54]:
print(num_sampled)
os.getcwd()
# os.chdir(cwd)

1


'C:\\Users\\apra\\Desktop\\SPRING 2020\\DS 4400\\ML1_Final_Proj'

In [55]:
def sample_n_new_images(n, to_dirname, meta_df, success_file_path, dead_file_path):
    """
    Samples and downloads n previously unsampled images from links in meta_df and saves them and their meta_data to a directory 
    with name to_dirname .Uses and updates successfully sampled indices and dead link indices in success_file_path and dead_file_path.
    
    """
    cwd = os.getcwd()
    #make directories to save sampled data
    to_dir = os.path.join(r'data',to_dirname)
    os.mkdir(to_dir)
    to_dir_images = os.path.join(to_dir,'images')
    os.mkdir(to_dir_images)
    

    
    successfully_sampled_indices = pd.read_csv(success_file_path, header = 0)
    dead_link_indices = pd.read_csv(dead_file_path, header = 0)

    
    #filter full dataframe to exclude above indices
    
    new_links = meta_df.drop(labels = successfully_sampled_indices['index'], axis='index')
    new_alive_links = new_links.drop(labels =  dead_link_indices['index'], axis = 'index')
    
    
    #randomly shuffle the filtered datframe
    num_good_rows = len(new_alive_links)
    n_sample = new_alive_links.sample(n=num_good_rows)
    
    num_successfully_sampled = 0
    #to update the indice lists for successfull and failed
    success_indices = successfully_sampled_indices['index'].tolist()
    dead_indices = dead_link_indices['index'].tolist()
    
    
    #sample metadata
    cleaned_sample = []
    
    #change directory to images directory
    os.chdir(to_dir_images)
    for idx, row in n_sample.iterrows():
        #break loop if we hit the number we want
        if(num_successfully_sampled == n):
            break
        url = row['link']
        try:
            r = requests.get(url, verify=False, timeout=2, stream=True)
            if r.status_code == 200:               
                file_name = str(idx)+'.png'
                with open(file_name, 'wb') as out_file:
                    #download the image and name it with its index in train.gcc
                    shutil.copyfileobj(r.raw, out_file)
                    
                img = Image.open(file_name) # open the image file
                img.verify() # verify that it is, in fact an image    
                #store the successfully sampled image's index
                success_indices.append(idx)
                num_successfully_sampled +=1
                #append the row to the meta_dataframe for this sample
                cleaned_sample.append(row.to_dict())
            else:
                print(f"failed to download image: {idx}")
                dead_indices.append(idx)

        except requests.exceptions.ReadTimeout:
            print (f"Read timed out for index: {idx}")
            
        except Exception as e:
            print(e)
            print (f"Other requests excpetion for index: {idx}")
            if(os.path.exists(file_name)):
                os.remove(file_name)
        except (IOError, SyntaxError) as e:
            print('Bad file:', idx) # print out the names of corrupt files
            if(os.path.exists(file_name)):
                os.remove(file_name)
    
    
    #add newly successful and failed links to respectful dataframes
    
            
    print(f"successfully sampled {num_successfully_sampled} new indices")
    
    #save the dataframes

    #samples metadata in parent
    os.chdir('..')
    sample_meta_df = pd.DataFrame.from_dict(cleaned_sample)
    sample_meta_df = sample_meta_df.set_index('index')
    sample_meta_df.to_csv('meta_data')



    #global sampled indices and dead links dataframe in project meta_data folder
    os.chdir(cwd)
    os.chdir(r'meta_data')
    
    
    
    
    
    successfully_sampled_indices_df = pd.Series(success_indices)
    successfully_sampled_indices_df.to_csv('sampled_idx_list',index = False, header = ['index'])

    dead_link_indice_df = pd.Series(dead_indices)
    dead_link_indice_df.to_csv('dead_link_idx_list', index=False, header = ['index'])


    #go back to cwd
    os.chdir(cwd)
    
        
        
    
    
    
    
    
    
    
    
    
    
    
    

In [56]:
sample_n_new_images(30000, r'30k_sample', train_gcc_meta, r'meta_data\sampled_idx_list', r'meta_data\dead_link_idx_list')

cannot identify image file '1759846.png'
Other requests excpetion for index: 1759846
cannot identify image file '1573065.png'
Other requests excpetion for index: 1573065
cannot identify image file '803514.png'
Other requests excpetion for index: 803514
failed to download image: 1597922
Read timed out for index: 1554794
failed to download image: 440076
cannot identify image file '961272.png'
Other requests excpetion for index: 961272
cannot identify image file '2004143.png'
Other requests excpetion for index: 2004143
failed to download image: 983555
cannot identify image file '402181.png'
Other requests excpetion for index: 402181
cannot identify image file '121948.png'
Other requests excpetion for index: 121948
cannot identify image file '1818400.png'
Other requests excpetion for index: 1818400
failed to download image: 645225
cannot identify image file '351947.png'
Other requests excpetion for index: 351947
cannot identify image file '821077.png'
Other requests excpetion for index: 82

cannot identify image file '1870662.png'
Other requests excpetion for index: 1870662
cannot identify image file '754216.png'
Other requests excpetion for index: 754216
cannot identify image file '1866434.png'
Other requests excpetion for index: 1866434
Read timed out for index: 1320437
cannot identify image file '515990.png'
Other requests excpetion for index: 515990
cannot identify image file '1444407.png'
Other requests excpetion for index: 1444407
Read timed out for index: 1175058
cannot identify image file '1262188.png'
Other requests excpetion for index: 1262188
Read timed out for index: 690757
cannot identify image file '388590.png'
Other requests excpetion for index: 388590
failed to download image: 48627
cannot identify image file '1586198.png'
Other requests excpetion for index: 1586198
failed to download image: 1913338
cannot identify image file '537880.png'
Other requests excpetion for index: 537880
failed to download image: 322569
cannot identify image file '377644.png'
Oth

failed to download image: 873183
cannot identify image file '738286.png'
Other requests excpetion for index: 738286
failed to download image: 1986865
cannot identify image file '962106.png'
Other requests excpetion for index: 962106
cannot identify image file '1206821.png'
Other requests excpetion for index: 1206821
cannot identify image file '1181449.png'
Other requests excpetion for index: 1181449
cannot identify image file '1987328.png'
Other requests excpetion for index: 1987328
cannot identify image file '1371027.png'
Other requests excpetion for index: 1371027
cannot identify image file '1966434.png'
Other requests excpetion for index: 1966434
failed to download image: 992059
cannot identify image file '281638.png'
Other requests excpetion for index: 281638
cannot identify image file '1264937.png'
Other requests excpetion for index: 1264937
failed to download image: 534473
failed to download image: 65206
cannot identify image file '634275.png'
Other requests excpetion for index: 

cannot identify image file '788051.png'
Other requests excpetion for index: 788051
failed to download image: 80997
cannot identify image file '646318.png'
Other requests excpetion for index: 646318
cannot identify image file '1052415.png'
Other requests excpetion for index: 1052415
failed to download image: 1895246
failed to download image: 1912018
failed to download image: 461845
failed to download image: 474741
failed to download image: 42701
failed to download image: 855455
Read timed out for index: 801176
cannot identify image file '903622.png'
Other requests excpetion for index: 903622
failed to download image: 205695
failed to download image: 1852100
Read timed out for index: 117708
cannot identify image file '336294.png'
Other requests excpetion for index: 336294
cannot identify image file '231925.png'
Other requests excpetion for index: 231925
failed to download image: 932398
failed to download image: 451180
failed to download image: 200942
cannot identify image file '445532.pn

failed to download image: 385839
cannot identify image file '370662.png'
Other requests excpetion for index: 370662
cannot identify image file '1830816.png'
Other requests excpetion for index: 1830816
failed to download image: 1057068
cannot identify image file '427861.png'
Other requests excpetion for index: 427861
Read timed out for index: 227668
cannot identify image file '1015956.png'
Other requests excpetion for index: 1015956
cannot identify image file '625672.png'
Other requests excpetion for index: 625672
failed to download image: 47442
failed to download image: 860531
failed to download image: 693802
failed to download image: 1462592
failed to download image: 503273
Read timed out for index: 1682156
Read timed out for index: 1303094
cannot identify image file '1694016.png'
Other requests excpetion for index: 1694016
Read timed out for index: 795077
failed to download image: 629170
cannot identify image file '1054380.png'
Other requests excpetion for index: 1054380
cannot ident

...

cannot identify image file '1938416.png'
Other requests excpetion for index: 1938416
cannot identify image file '1167372.png'
Other requests excpetion for index: 1167372
failed to download image: 1047118
cannot identify image file '1227799.png'
Other requests excpetion for index: 1227799
cannot identify image file '1773612.png'
Other requests excpetion for index: 1773612
cannot identify image file '1758212.png'
Other requests excpetion for index: 1758212
failed to download image: 1499573
cannot identify image file '318452.png'
Other requests excpetion for index: 318452
cannot identify image file '117652.png'
Other requests excpetion for index: 117652
cannot identify image file '1197856.png'
Other requests excpetion for index: 1197856
failed to download image: 1802904
failed to download image: 1438729
failed to download image: 1282949
cannot identify image file '697954.png'
Other requests excpetion for index: 697954
failed to download image: 646767
HTTPConnectionPool(host='images.slidep

failed to download image: 214099
cannot identify image file '1612763.png'
Other requests excpetion for index: 1612763
cannot identify image file '1588533.png'
Other requests excpetion for index: 1588533
cannot identify image file '1813015.png'
Other requests excpetion for index: 1813015
cannot identify image file '561125.png'
Other requests excpetion for index: 561125
cannot identify image file '358856.png'
Other requests excpetion for index: 358856
cannot identify image file '387469.png'
Other requests excpetion for index: 387469
failed to download image: 1126288
cannot identify image file '1392488.png'
Other requests excpetion for index: 1392488
cannot identify image file '364468.png'
Other requests excpetion for index: 364468
Read timed out for index: 1482053
cannot identify image file '1027437.png'
Other requests excpetion for index: 1027437
cannot identify image file '1222490.png'
Other requests excpetion for index: 1222490
Read timed out for index: 1099208
Read timed out for inde

failed to download image: 657941
cannot identify image file '574198.png'
Other requests excpetion for index: 574198
failed to download image: 1267025
failed to download image: 1605604
cannot identify image file '659600.png'
Other requests excpetion for index: 659600
failed to download image: 403806
cannot identify image file '48999.png'
Other requests excpetion for index: 48999
cannot identify image file '799772.png'
Other requests excpetion for index: 799772
failed to download image: 503725
cannot identify image file '1549983.png'
Other requests excpetion for index: 1549983
cannot identify image file '223158.png'
Other requests excpetion for index: 223158
failed to download image: 401946
cannot identify image file '1951230.png'
Other requests excpetion for index: 1951230
failed to download image: 195825
Read timed out for index: 1128416
Read timed out for index: 1711882
failed to download image: 1641076
failed to download image: 1704107
cannot identify image file '49880.png'
Other req

cannot identify image file '1727217.png'
Other requests excpetion for index: 1727217
cannot identify image file '1661833.png'
Other requests excpetion for index: 1661833
failed to download image: 432764
failed to download image: 860682
cannot identify image file '1621381.png'
Other requests excpetion for index: 1621381
cannot identify image file '1700884.png'
Other requests excpetion for index: 1700884
Read timed out for index: 1257501
failed to download image: 579403
failed to download image: 1730589
cannot identify image file '1308789.png'
Other requests excpetion for index: 1308789
cannot identify image file '1779593.png'
Other requests excpetion for index: 1779593
failed to download image: 261006
failed to download image: 1138837
cannot identify image file '1693899.png'
Other requests excpetion for index: 1693899
failed to download image: 324738
cannot identify image file '515649.png'
Other requests excpetion for index: 515649
cannot identify image file '246038.png'
Other requests 

HTTPSConnectionPool(host='www.classicchicagomagazine.com', port=443): Max retries exceeded with url: /wp-content/uploads/sites/2472/2017/02/A-sandy-path-right-to-the-ocean-e1486494560183.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000024B32284A30>, 'Connection to www.classicchicagomagazine.com timed out. (connect timeout=2)'))
Other requests excpetion for index: 289761
failed to download image: 808224
failed to download image: 268637
failed to download image: 1256738
cannot identify image file '437042.png'
Other requests excpetion for index: 437042
failed to download image: 1923366
cannot identify image file '878264.png'
Other requests excpetion for index: 878264
cannot identify image file '332018.png'
Other requests excpetion for index: 332018
cannot identify image file '896955.png'
Other requests excpetion for index: 896955
failed to download image: 189176
cannot identify image file '1969208.png'
Other requests excpetion for index: 1969208
cann

In [50]:
# sampe_n_new_images(100, r'100_sample_2', train_gcc_meta, r'meta_data\sampled_idx_list', r'meta_data\dead_link_idx_list')

cannot identify image file '923320.png'
Other requests excpetion for index: 923320
failed to download image: 1467632
cannot identify image file '1046783.png'
Other requests excpetion for index: 1046783
failed to download image: 1114710
cannot identify image file '1246139.png'
Other requests excpetion for index: 1246139
failed to download image: 999758
failed to download image: 1097336
successfully sampled 100 new indices


In [90]:
# #now to verify the downloaded data and see if any were not downloaded properly

# def verify_downloads(down_dirname):
#     cwd = os.getcwd()
    
#    #find dirmname
#     down_dir = os.path.join(r'data',down_dirname)
#     down_dir_images = os.path.join(down_dir,'images')
#     down_dir_meta = os.path.join(down_dir, 'meta_data')
#     cleaned_meta_path = os.path.join(down_dir, 'cleaned_meta_data')
#     bad_files_path = os.path.join(down_dir, 'bad_image_indices')
   

#     sample_df = pd.read_csv(down_dir_meta, index_col='index')

#     good_count = 0
#     badFiles = []
#     badNumbers = []
    
#     print(len(sample_df))


#     for filename in os.listdir(down_dir_images):
#         if filename.endswith('.png'):
#             try:
#                 file_path = os.path.join(down_dir_images, filename)
#                 img = Image.open(file_path) # open the image file
#                 img.verify() # verify that it is, in fact an image
#                 good_count+=1
#             except (IOError, SyntaxError) as e:
#                 number = os.path.splitext(filename)[0]
#                 badNumbers.append(int(number))
#                 badFiles.append(filename)
#                 print('Bad file:', filename) # print out the names of corrupt files
                

  
#     for bad in badFiles:    
#         os.remove(os.path.join(down_dir_images,  bad))

    
#     cleaned_meta_df = sample_df.drop(labels = badNumbers, axis='index')
#     cleaned_meta_df.to_csv(cleaned_meta_path)
    
#     bad_indices_df = pd.Series(badNumbers)
#     bad_indices_df.to_csv(bad_files_path,index = False, header = ['index'])
    
#     print("Number of workable images: ", good_count)
    

20002
Bad file: 1006951.png
Bad file: 1007762.png
Bad file: 1010583.png
Bad file: 1012717.png
Bad file: 1013445.png
Bad file: 1018792.png
Bad file: 1022543.png
Bad file: 1024948.png
Bad file: 1029012.png
Bad file: 103089.png
Bad file: 1034828.png
Bad file: 1035169.png
Bad file: 1042929.png
Bad file: 1043065.png
Bad file: 1045449.png
Bad file: 1048066.png
Bad file: 1048214.png
Bad file: 105067.png
Bad file: 1053247.png
Bad file: 1055913.png
Bad file: 1058284.png
Bad file: 106226.png
Bad file: 1064937.png
Bad file: 1065785.png
Bad file: 1096882.png
Bad file: 1101031.png
Bad file: 1113691.png
Bad file: 1115702.png
Bad file: 1134195.png
Bad file: 1142352.png
Bad file: 1142386.png
Bad file: 1143249.png
Bad file: 1145298.png
Bad file: 1157305.png
Bad file: 1161021.png
Bad file: 1161151.png
Bad file: 1161292.png
Bad file: 1163998.png
Bad file: 1166245.png
Bad file: 1170375.png
Bad file: 1173401.png
Bad file: 1173627.png
Bad file: 1175244.png
Bad file: 1177796.png
Bad file: 1178884.png
Bad fil

Bad file: 245442.png
Bad file: 2460742.png
Bad file: 2467758.png
Bad file: 246982.png
Bad file: 2471230.png
Bad file: 2473687.png
Bad file: 2479699.png
Bad file: 2483970.png
Bad file: 2486319.png
Bad file: 2489413.png
Bad file: 2491707.png
Bad file: 2493379.png
Bad file: 2496963.png
Bad file: 2503933.png
Bad file: 2512047.png
Bad file: 2516421.png
Bad file: 2516571.png
Bad file: 2521978.png
Bad file: 2522389.png
Bad file: 2526619.png
Bad file: 2529175.png
Bad file: 2531553.png
Bad file: 2536168.png
Bad file: 2551833.png
Bad file: 2553655.png
Bad file: 2556630.png
Bad file: 2556770.png
Bad file: 2558982.png
Bad file: 2561534.png
Bad file: 2562550.png
Bad file: 256298.png
Bad file: 256382.png
Bad file: 2565449.png
Bad file: 256782.png
Bad file: 2572979.png
Bad file: 2581164.png
Bad file: 2583986.png
Bad file: 2584162.png
Bad file: 2585330.png
Bad file: 2586168.png
Bad file: 2586511.png
Bad file: 2591085.png
Bad file: 2592477.png
Bad file: 2604562.png
Bad file: 2604784.png
Bad file: 26059

Bad file: 956110.png
Bad file: 962298.png
Bad file: 963135.png
Bad file: 965910.png
Bad file: 970188.png
Bad file: 970482.png
Bad file: 97908.png
Bad file: 981153.png
Bad file: 981415.png
Bad file: 983635.png
Bad file: 984860.png
Bad file: 99360.png
[1006951, 1007762, 1010583, 1012717, 1013445, 1018792, 1022543, 1024948, 1029012, 103089, 1034828, 1035169, 1042929, 1043065, 1045449, 1048066, 1048214, 105067, 1053247, 1055913, 1058284, 106226, 1064937, 1065785, 1096882, 1101031, 1113691, 1115702, 1134195, 1142352, 1142386, 1143249, 1145298, 1157305, 1161021, 1161151, 1161292, 1163998, 1166245, 1170375, 1173401, 1173627, 1175244, 1177796, 1178884, 1187642, 1193658, 1196729, 1203132, 1214279, 1219056, 1224248, 1226395, 122879, 1233545, 1235377, 1236175, 1238872, 1248389, 1248995, 1249897, 1254416, 1254725, 1258460, 1263839, 1271336, 1273476, 1278753, 1283682, 1284864, 1286139, 1289842, 1292832, 1296876, 1299070, 1300790, 130614, 130971, 1313631, 1314615, 1324787, 1329291, 1333240, 134703, 